## Main structure of CLOB
- deal with each order and try to match a successful trade
- search the ideal prices, from good to bad, then from ealry to late
- place in a proper position if not matched / partly matched
- update the CLOB
- record in the trade book (trade_df)

In [163]:
import logging
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Configure logging at the top level
log_path_to_clob = "clob.log"
logger = logging.getLogger('CLOBLogger')
logger.setLevel(logging.INFO)

# Prevent adding multiple handlers if the logger already has handlers
if not logger.handlers:
    file_handler = logging.FileHandler(log_path_to_clob, mode='w')
    formatter = logging.Formatter('%(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

class CLOB():
    def __init__(self, orders):
        """
        Initialize the CLOB with given orders.
        
        Args:
            orders (pd.DataFrame): DataFrame containing order details.
        """
        # initialize orders and CLOB
        self.orders = orders
        self.clob = {"buy": {}, "sell": {}}
        # market metrics
        self.passed_time = 0
        self.spread_dollars = 0
        self.market_depth = 0
        self.spread_ticks = 0
        self.spread_bps = 0
        # trade and aggregation data structures
        self.trade_df = pd.DataFrame(columns=["trade_time", "buy_group_id", "sell_group_id", "price", "size"])
        self.new_df = pd.DataFrame(columns=self.trade_df.columns)
        self.aggregated_clob = pd.DataFrame(columns=["Size", "Bid Price", "Ask Price", "Size"])
    
    def restart(self):
        """Reset CLOB/metrics/trade data"""
        self.clob = {"buy": {}, "sell": {}}

        self.passed_time = 0
        self.spread_dollars = 0
        self.market_depth = 0
        self.spread_ticks = 0
        self.spread_bps = 0

        self.trade_df = pd.DataFrame(columns=["trade_time", "buy_group_id", "sell_group_id", "price", "size"])
        self.new_df = pd.DataFrame(columns=self.trade_df.columns)
        self.aggregated_clob = pd.DataFrame(columns=["Size", "Bid Price", "Ask Price", "Size"])

    def print_aggregated_clob(self):
        """Display the current aggregated state of the order book (buy/sell)."""
        self.aggregated_clob = pd.DataFrame(columns=["Size", "Bid Price", "Ask Price", "Size"])

        log_message =  "-"*60 + "\n"
        log_message += " "*15 + "Current Aggregated CLOB state: \n"
        log_message += "-"*60 + "\n"
        log_message += "{:<16} {:<16} {:<16} {:<16}\n".format("Size", "Bid Price", "Ask Price", "Size")
        # aggregate buy and sell orders
        buy_orders = [(price, sum(order['size'] for order in orders)) for price, orders in self.clob["buy"].items() if self.has_size_given_price(price, "buy")]
        sell_orders = [(price, sum(order['size'] for order in orders)) for price, orders in self.clob["sell"].items() if self.has_size_given_price(price, "sell")]
        # sort buy orders (descending by price) and sell orders (ascending by price)
        buy_orders.sort(key=lambda x: x[0], reverse=True)
        sell_orders.sort(key=lambda x: x[0])

        max_rows = max(len(buy_orders), len(sell_orders))

        for i in range(max_rows):
            buy_price, buy_size = buy_orders[i] if i < len(buy_orders) else ("", "")
            sell_price, sell_size = sell_orders[i] if i < len(sell_orders) else ("", "")
            log_message += "{:<16} {:<16} {:<16} {:<16}\n".format(buy_size, f"{buy_price:.2f}" if buy_price else "", f"{sell_price:.2f}" if sell_price else "", sell_size)
            self.aggregated_clob = pd.concat([self.aggregated_clob, pd.DataFrame([[buy_size, buy_price, sell_price, sell_size]], columns=self.aggregated_clob.columns)], ignore_index=True)

        log_message += "-"*60
        log_message += "\n"*3
        print(log_message)


    def print_top_orders(self):
        """Print the highest bid and lowest ask orders."""
        highest_bid = max([p for p in self.clob["buy"].keys() if self.has_size_given_price(p, "buy")]) if self.clob["buy"] else None
        lowest_ask = min([p for p in self.clob["sell"].keys() if self.has_size_given_price(p, "sell")]) if self.clob["sell"] else None
        

        log_message = "-"*80 + "\n"
        log_message += " "*35 + "Top Orders: \n"
        log_message += "-"*80 + "\n"
        
        if highest_bid:
            log_message += f"Highest Bid: {round(highest_bid, 2):.2f}\n"
            for order in self.clob["buy"][highest_bid]:
                if order['size'] > 0:
                    log_message += "  Order ID: {:<4} Group ID: {:<4} Size: {:<4} Time: {}\n".format(
                        order['order id'], order['group id'],order['size'], order['time'])
        else:
            log_message += "No bids available.\n"
        
        if lowest_ask:
            log_message += f"Lowest Ask: {round(lowest_ask, 2):.2f}\n"
            for order in self.clob["sell"][lowest_ask]:
                if order['size'] > 0:
                    log_message += "  Order ID: {:<4} Group ID: {:<4} Size: {:<4} Time: {}\n".format(
                        order['order id'], order['group id'],order['size'], order['time'])
        else:
            log_message += "No asks available.\n"
        
        log_message += "-"*80
        log_message += "\n"*3
        print(log_message)

    def print_trade(self):
        """Print the current trade state."""
        log_message = "-"*80 + "\n"
        log_message += " "*30 + "Current Trade state: \n"
        log_message += "-"*80 + "\n"
        if self.trade_df.empty:
            log_message += "No trade happened"
        else:
            log_message += self.trade_df.to_string()
        log_message += "\n"
        log_message += "-"*80
        log_message += "\n"*3
        print(log_message)


    def log_clob(self, is_log=False,is_print=False):
        """Log the current CLOB state."""
        log_message =  "-"*80 + "\n"
        log_message += " "*30 + "Current CLOB state: \n"
        log_message += "-"*80 + "\n"
        for side in ["buy", "sell"]:
            log_message += "Bid:\n" if side == "buy" else "Ask:\n"
            for price, orders in self.clob[side].items():
                if self.has_size_given_price(price, side):
                    log_message += f"Price: {price:.2f}\n"
                    for order in orders:
                        if order['size'] > 0:
                            log_message += "  Order ID: {:<4} Group ID: {:<4} Size: {:<4} Time: {}\n".format(
                                order['order id'], order.get('group id', 'N/A'), order['size'], order['time']
                            )
            if side == "buy":
                log_message += "\n" 
        log_message += "-"*80
        if is_print:
            print(log_message)
        if is_log:
            logger.info(log_message)

    def log_trade(self,is_log=False):
        """Log the details of all trades matched."""
        log_message = "-"*80 + "\n"
        log_message += " "*30 + "Current Trade Done: \n"
        log_message += "-"*80 + "\n"
        if self.new_df.empty:
            log_message += "No trade happened"
        else:
            log_message += self.new_df.to_string()
        self.new_df = pd.DataFrame(columns=self.trade_df.columns)
        if is_log:
            logger.info(log_message)

    def upload_clob_info(self, order, outstanding=""):
        """
        Add order to the CLOB.

        Args:
            order (dict): Order details.
            outstanding (int, optional): Outstanding size. Defaults to order's size.
        """
        # add order to the CLOB, where the outstanding size is the size after trading (if matched)
        if not outstanding:
            outstanding = order["size"]
        if type(outstanding) != int:
            outstanding = int(outstanding)
        price0 = float(order["price"])

        # every order with the outstanding left will be recorded in the clob dictionary, key is its size & price
        order_dic = {
            "time":order["timestamp"],
            "order id": int(order["order id"]), 
            "size": int(outstanding),
            "group id": int(order["group id"])
        }
        # add the order to the end of its price list, as it's the latest order
        if price0 not in self.clob[order["side"]]:
            self.clob[order["side"]][price0] = [order_dic]
        else:
            self.clob[order["side"]][price0].append(order_dic)
        # sort CLOB
        if order["side"] == "buy":
            self.clob["buy"] = dict(sorted(self.clob["buy"].items(), reverse=True))
        else:
            self.clob["sell"] = dict(sorted(self.clob["sell"].items()))

    def upload_trade_info(self, trades):
        """
        Record the details of all trades matched.

        Args:
            trades (list): List of matched trade dictionaries.
        """
        for trade in trades:
            # every successful trade will be recorded in the trade_df
            buyer_seller = {self.orders.iloc[trade["latecomer"]-1, 2]: self.orders.iloc[trade["latecomer"]-1, 1],
                            self.orders.iloc[trade["pre-existing"]-1, 2]: self.orders.iloc[trade["pre-existing"]-1, 1]}
            trade_detail =  [
                self.orders.iloc[trade["latecomer"]-1, 0], # timestamp
                buyer_seller["buy"], # buy_group_id
                buyer_seller["sell"], # sell_group_id
                self.orders.iloc[trade["pre-existing"]-1, 3], # price
                trade["trade size"]] # size
            # add to the end of the trade_df, as it's the latest order
            self.new_df = pd.concat([self.new_df, pd.DataFrame([trade_detail], columns=self.new_df.columns)], ignore_index=True)
        self.trade_df = pd.concat([self.trade_df, self.new_df], ignore_index=True)
        
    def has_size_given_price(self, price, side):
        """
        Check if there is any outstanding size for a given price and side.

        Args:
            price (float): The price level.
            side (str): 'buy' or 'sell'.

        Returns:
            bool: True if there is outstanding size, False otherwise.
        """
        price = float(price)
        try:
            order_sizes = [order['size'] for order in self.clob[side][price]]
            return sum(order_sizes) > 0
        except KeyError:
            print("Error: no such price")
            return False
        
    def match_trade(self, order):
        """
        Match an incoming order with existing orders in the CLOB.

        Args:
            order (dict): Incoming order details.

        Returns:
            tuple: (outstanding_size, matched_trades)
        """
        matched_trades = []

        cur_side, cur_price, cur_outstanding = order["side"], float(order["price"]), int(order["size"])
        opp_side = "buy" if cur_side == "sell" else "sell"
        opp_side_price = self.clob[opp_side].keys()

        # get the good sell prices that are good for the current buy order
        if cur_side == "buy":
            # Sell prices <= Buy price
            good_prices = [p for p in opp_side_price if
                           p <= cur_price and self.has_size_given_price(p, opp_side)]
            good_prices.sort() # Lowest price first
        else:
            # Buy prices >= Sell price
            good_prices = [p for p in opp_side_price if 
                           p >= cur_price and self.has_size_given_price(p, opp_side)]
            good_prices.sort(reverse=True) # Highest price first

        if not good_prices:
            return cur_outstanding, matched_trades

        for good_price in good_prices:
            for i, opp_order in enumerate(self.clob[opp_side][good_price]):
                opp_size = opp_order["size"]
                if opp_size == 0:
                    continue
                if opp_size >= cur_outstanding:
                    matched_trades.append({"pre-existing":int(opp_order["order id"]),
                                            "latecomer": int(order["order id"]),
                                            "trade size": cur_outstanding})
                    self.clob[opp_side][good_price][i]["size"] -= cur_outstanding # assign the value in the dictionary
                    return 0, matched_trades
                else:
                    cur_outstanding -= opp_size
                    self.clob[opp_side][good_price][i]["size"] = 0 # assign the value in the dictionary
                    matched_trades.append({"pre-existing":int(opp_order["order id"]),
                                            "latecomer": int(order["order id"]),
                                            "trade size": opp_size})
        return cur_outstanding, matched_trades
    
    def get_bid_ask_metrix(self,order):
        """
        Update the bid-ask spread value, spread in ticks, spread in bps, and average market depth.

        bid-ask spread: the difference between the highest bid price and the lowest ask price.
        spread in ticks: the spread in dollars divided by the tick size ($0.01).
        spread in bps: the spread in dollars divided by the highest bid price and multiplied by 10,000.
        average market depth: the average of the best bid value and the best ask value.
        """
        
        highest_bid = max([p for p in self.clob["buy"].keys() if self.has_size_given_price(p, "buy")]) if self.clob["buy"] else None
        lowest_ask = min([p for p in self.clob["sell"].keys() if self.has_size_given_price(p, "sell")]) if self.clob["sell"] else None
        
        if highest_bid and lowest_ask:
            # count the spread(this cycle)
            spread_dollars = lowest_ask - highest_bid
            # count the average market depth(this cycle)
            best_bid_quantity = sum(order['size'] for order in self.clob["buy"][highest_bid])
            best_ask_quantity = sum(order['size'] for order in self.clob["sell"][lowest_ask])
            best_bid_value = best_bid_quantity * highest_bid
            best_ask_value = best_ask_quantity * lowest_ask
            avg_market_depth = (best_bid_value + best_ask_value) / 2

            # time weighted average spread and market depth
            self.spread_dollars = self.spread_dollars * self.passed_time + spread_dollars * order["delta_time_sec"]
            self.market_depth = self.market_depth * self.passed_time + avg_market_depth * order["delta_time_sec"]
            # update time
            self.passed_time += order["delta_time_sec"]
            self.spread_dollars = self.spread_dollars / self.passed_time
            self.market_depth = self.market_depth / self.passed_time
            
            # calculate the spread in ticks and bps
            self.spread_ticks = self.spread_dollars / 0.01  # 每个tick为$0.01
            self.spread_bps = (self.spread_dollars / highest_bid) * 10000  # 基点计算
        
    def step_run(self, order, is_log=False,is_print_step=True,is_print_matrix=True):
        """
        Process a single order.

        Args:
            order (dict): Order details.
            is_log (bool, optional): If True, log detailed information. Defaults to False.
        """
        outstanding, matched_trades = self.match_trade(order)

        if matched_trades:
            self.upload_trade_info(matched_trades)
        if outstanding > 0:
            self.upload_clob_info(order, outstanding)
        
        self.get_bid_ask_metrix(order)
        self.log_trade(is_log)
        self.log_clob(is_log)
       
        if is_print_step:
            print(f"Coming Order ID {order['order id']}: {order['side']} {order['size']} @ {order['price']} by Group {order['group id']}")
        if is_print_matrix:
            print(f"Spread: ${self.spread_dollars:.2f}, Ticks: {self.spread_ticks:.2f}, BPS: {self.spread_bps:.2f}, Average Market Depth: ${self.market_depth:.2f}")
        if is_log:
            logger.info(f"Coming Order ID {order['order id']}: {order['side']} {order['size']} @ {order['price']} by Group {order['group id']}\nSpread: ${self.spread_dollars:.2f}, Ticks: {self.spread_ticks:.2f}, BPS: {self.spread_bps:.2f}, Average Market Depth: ${self.market_depth:.2f}")
            logger.info("-"*80+"\n"*3)

    def run(self, is_log=False,is_print_top=False,is_print_aggregated=False,is_print_trade=False,is_print_step=True,is_print_matrix=False): 
        """Run the CLOB simulation for all orders."""
        for _, order in self.orders.iterrows():
            self.step_run(order,is_log,is_print_step,is_print_matrix)
            if is_print_top:
                self.print_top_orders()
            if is_print_aggregated:
                self.print_aggregated_clob()
            if is_print_trade:
                self.print_trade()
            if is_log:
                print('Done. Please check the log file (clob.log) for the details of the CLOB and trades.') 

# load and ini data

In [164]:
import pandas as pd

orders = pd.read_csv('orders.csv')
orders["Timestamp"] = pd.to_datetime(orders["Timestamp"])

orders["delta_time_sec"] = (orders["Timestamp"].shift(-1) - orders["Timestamp"]).dt.total_seconds().fillna(0)

print(orders.head())
print(orders.tail())

orders.columns = ['timestamp', 'group id', 'side', 'price', 'size','delta_time_sec']

orders["group id"] = orders["group id"].astype('int')
orders["price"] = orders["price"].astype('float')
orders["size"] = orders["size"].astype('int')

orders["order id"] = orders.index + 1
print(orders.head())
print(orders.describe())

clob = CLOB(orders)

            Timestamp  Group ID  Side  Price  Size  delta_time_sec
0 2024-09-30 20:26:01         5   buy    8.0    50            10.0
1 2024-09-30 20:26:11        14  sell   11.0    20             3.0
2 2024-09-30 20:26:14        10  sell   11.0   100             1.0
3 2024-09-30 20:26:15        12  sell   10.0    50             1.0
4 2024-09-30 20:26:16         7   buy    5.0    50             4.0
             Timestamp  Group ID  Side  Price  Size  delta_time_sec
95 2024-09-30 20:42:24        12  sell   9.50    15             0.0
96 2024-09-30 20:42:24         9   buy   9.10   250            17.0
97 2024-09-30 20:42:41         9   buy   9.60   500            19.0
98 2024-09-30 20:43:00        20  sell  13.00    60            14.0
99 2024-09-30 20:43:14        10  sell  11.01    10             0.0
            timestamp  group id  side  price  size  delta_time_sec  order id
0 2024-09-30 20:26:01         5   buy    8.0    50            10.0         1
1 2024-09-30 20:26:11        14  sel

## Order Book State

In [165]:
clob.run(is_log=False,is_print_top=True)

Coming Order ID 1: buy 50 @ 8.0 by Group 5
--------------------------------------------------------------------------------
                                   Top Orders: 
--------------------------------------------------------------------------------
Highest Bid: 8.00
  Order ID: 1    Group ID: 5    Size: 50   Time: 2024-09-30 20:26:01
No asks available.
--------------------------------------------------------------------------------



Coming Order ID 2: sell 20 @ 11.0 by Group 14
--------------------------------------------------------------------------------
                                   Top Orders: 
--------------------------------------------------------------------------------
Highest Bid: 8.00
  Order ID: 1    Group ID: 5    Size: 50   Time: 2024-09-30 20:26:01
Lowest Ask: 11.00
  Order ID: 2    Group ID: 14   Size: 20   Time: 2024-09-30 20:26:11
--------------------------------------------------------------------------------



Coming Order ID 3: sell 100 @ 11.0 by Group

- Implement the function to print the full order book for both bid ask price levels with aggregated size for each level. (1.5%)

In [166]:
clob.restart()
clob.run(is_log=False,is_print_aggregated=True)
clob.aggregated_clob.to_csv("final_clob.csv", index=False)

Coming Order ID 1: buy 50 @ 8.0 by Group 5
------------------------------------------------------------
               Current Aggregated CLOB state: 
------------------------------------------------------------
Size             Bid Price        Ask Price        Size            
50               8.00                                              
------------------------------------------------------------



Coming Order ID 2: sell 20 @ 11.0 by Group 14
------------------------------------------------------------
               Current Aggregated CLOB state: 
------------------------------------------------------------
Size             Bid Price        Ask Price        Size            
50               8.00             11.00            20              
------------------------------------------------------------



Coming Order ID 3: sell 100 @ 11.0 by Group 10
------------------------------------------------------------
               Current Aggregated CLOB state: 
------------------

# Show the order book state after 5th order, 20th order, and last order.

In [167]:
clob.restart()
index = [4,19,len(orders)-1]
for j, order in orders.iterrows():
    clob.step_run(order, is_log=False,is_print_step=False,is_print_matrix=False)
    if j in index:
        print(f"Order {j+1}")
        clob.print_aggregated_clob()

Order 5
------------------------------------------------------------
               Current Aggregated CLOB state: 
------------------------------------------------------------
Size             Bid Price        Ask Price        Size            
50               8.00             10.00            50              
50               5.00             11.00            120             
------------------------------------------------------------



Order 20
------------------------------------------------------------
               Current Aggregated CLOB state: 
------------------------------------------------------------
Size             Bid Price        Ask Price        Size            
90               10.00            11.00            110             
150              8.00             12.00            100             
142              7.00             12.50            20              
50               5.00             15.00            40              
                                  30.

In [168]:
clob.restart()
index = [4,19,len(orders)-1]
for j, order in orders.iterrows():
    clob.step_run(order, is_log=False,is_print_step=False,is_print_matrix=False)
    if j in index:
        print(f"Order {j+1}")
        clob.log_clob(False,True)

Order 5
--------------------------------------------------------------------------------
                              Current CLOB state: 
--------------------------------------------------------------------------------
Bid:
Price: 8.00
  Order ID: 1    Group ID: 5    Size: 50   Time: 2024-09-30 20:26:01
Price: 5.00
  Order ID: 5    Group ID: 7    Size: 50   Time: 2024-09-30 20:26:16

Ask:
Price: 10.00
  Order ID: 4    Group ID: 12   Size: 50   Time: 2024-09-30 20:26:15
Price: 11.00
  Order ID: 2    Group ID: 14   Size: 20   Time: 2024-09-30 20:26:11
  Order ID: 3    Group ID: 10   Size: 100  Time: 2024-09-30 20:26:14
--------------------------------------------------------------------------------
Order 20
--------------------------------------------------------------------------------
                              Current CLOB state: 
--------------------------------------------------------------------------------
Bid:
Price: 10.00
  Order ID: 15   Group ID: 3    Size: 40   Time: 202

## Trade Log
- Record the details of all trades matched with all the necessary information (2%).

In [169]:
clob.restart()
clob.run(is_log=False,is_print_trade=True)
clob.trade_df["trade_time"] = clob.trade_df["trade_time"].astype(str)
clob.trade_df.to_excel("final_trade.xlsx", index=False)

Coming Order ID 1: buy 50 @ 8.0 by Group 5
--------------------------------------------------------------------------------
                              Current Trade state: 
--------------------------------------------------------------------------------
No trade happened
--------------------------------------------------------------------------------



Coming Order ID 2: sell 20 @ 11.0 by Group 14
--------------------------------------------------------------------------------
                              Current Trade state: 
--------------------------------------------------------------------------------
No trade happened
--------------------------------------------------------------------------------



Coming Order ID 3: sell 100 @ 11.0 by Group 10
--------------------------------------------------------------------------------
                              Current Trade state: 
--------------------------------------------------------------------------------
No trade happened


# Spread and Market Depth

In [170]:
clob.restart()
clob.run(is_log=True,is_print_top=True,is_print_matrix=True)

Coming Order ID 1: buy 50 @ 8.0 by Group 5
Spread: $0.00, Ticks: 0.00, BPS: 0.00, Average Market Depth: $0.00
--------------------------------------------------------------------------------
                                   Top Orders: 
--------------------------------------------------------------------------------
Highest Bid: 8.00
  Order ID: 1    Group ID: 5    Size: 50   Time: 2024-09-30 20:26:01
No asks available.
--------------------------------------------------------------------------------



Done. Please check the log file (clob.log) for the details of the CLOB and trades.
Coming Order ID 2: sell 20 @ 11.0 by Group 14
Spread: $3.00, Ticks: 300.00, BPS: 3750.00, Average Market Depth: $310.00
--------------------------------------------------------------------------------
                                   Top Orders: 
--------------------------------------------------------------------------------
Highest Bid: 8.00
  Order ID: 1    Group ID: 5    Size: 50   Time: 2024-09-30